# 1. ETL ➡ Extraer, Transformar y Cargar

Obtengo los datos a través del cliente interesado en que se le analicen los datos de su negocio. En este caso, es una farmacia comunitaria de Canarias, en Tenerife.
 
El cliente ha obtenido los datos a partir del software de su negocio, llamado [Unycop](https://www.unycop.es/). Los datos se encuentran en formato excel.

In [289]:
#%pip install xlrd>=2.0.1

In [290]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import funciones

ModuleNotFoundError: No module named 'funciones'

---

### Limpieza del archivo excel articulos

In [ ]:
df = pd.read_excel('../data/data_raw/articulos.xlsx')
df = df.iloc[:, 1:]
df.columns = df.iloc[0]
df = df.iloc[1:, :]
df = df.reset_index(drop=True)


In [ ]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,NaN,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53",NaN,"18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68",NaN,"18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25",NaN,"16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22",NaN,"15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14",NaN,"14.966,80","0,66%"


In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.rename(columns={df.columns[6]: 'columna_vacia'})
columna_vacia = 'columna_vacia'
df = df.drop(columns=[columna_vacia], errors='ignore')


In [ ]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53","18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68","18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25","16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22","15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14","14.966,80","0,66%"


In [ ]:
df = df.dropna()

In [ ]:
df.head()

,Código,Nombre,Familia,Mínimo,Existencias,P.V.P.,Ventas,Porcent.
0,702686,JARDIANCE 10 MG 30 COMP,Financiables,4,5,"49,53","18.188,36","0,80%"
1,706414,TOUJEO 300 UI SOL INYECT 3 PLUMAS 1.5 M,Financiables,5,6,"48,68","18.011,60","0,80%"
2,700588,FORXIGA 10 MG 28 COMP,Financiables,3,3,"44,25","16.149,68","0,71%"
3,704632,TRULICITY 1.5 MG SOL INY 4 PLUMAS PREC,Financiables,2,2,"123,22","15.895,38","0,70%"
4,935353,NEXIUM MUPS 40 MG 28 COMP,Financiables,5,4,"24,14","14.966,80","0,66%"


In [ ]:
columnas_a = ['Código', 'Nombre', 'Familia', 'Mínimo', 'Existencias', 'P.V.P.', 'Ventas', 'Porcent.']
eliminar = ['Código', 'Nombre', 'Familia', 'Mínimo', 'Existencias', 'P.V.P.', 'Ventas', 'Porcent.']

for columna, valor in zip(columnas_a, eliminar):
    df = df[~(df[columna] == valor)]

In [ ]:
df.to_excel('../data/data_clean/archivos.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: '../data/data_clean/archivos.xlsx'

- Ánalisis del primer archivo

In [ ]:
df.shape

(3352, 8)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3352 entries, 0 to 3436
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Código       3352 non-null   object
 1   Nombre       3352 non-null   object
 2   Familia      3352 non-null   object
 3   Mínimo       3352 non-null   object
 4   Existencias  3352 non-null   object
 5   P.V.P.       3352 non-null   object
 6   Ventas       3352 non-null   object
 7   Porcent.     3352 non-null   object
dtypes: object(8)
memory usage: 235.7+ KB


In [ ]:
df.describe().T

,count,unique,top,freq
0,,,,
Código,3352,3352,702686,1
Nombre,3352,3349,OMEPRAZOL STADA 20 MG 56 CAPS EFG,2
Familia,3352,9,Financiables,1652
Mínimo,3352,18,1,1371
Existencias,3352,89,1,1056
P.V.P.,3352,1551,"2,40",71
Ventas,3352,3052,"156,00",6
Porcent.,3352,48,"0,01%",1580


---

### Limpieza del archivo lineas_ventas_diciembre_2017 en excel 

In [ ]:
dic_17 = pd.read_excel('../data/data_raw/2017/lineas_ventas_diciembre_17.xls')

In [ ]:
dic_17.head(5)

,Unnamed: 0,Fecha,Hora,Tipo de Operación,Empresa,Código,Denominación,Organismo,Cantidad (Unidades),Pvp,PVP Facturado,Importe Bruto,Descuento,Importe Neto,Cliente,Perfil de Cliente,Vendedor,Aporta.Subv.,Número de Puesto,Existencias Anteriores,Existencias Posteriores
0,NaN,01/12/17,08:10,Contado,1,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,091 - TSI 002,1,1.65,1.65,0.17,0.0,0.17,NaN,NaN,ADA,0,1,36,35
1,NaN,01/12/17,08:10,Contado,1,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,095 - RP RE DOE,1,1.65,1.65,0.00,0.0,0.00,NaN,NaN,ADA,0,1,37,36
2,NaN,01/12/17,08:10,Contado,1,695435,VIBRACINA 100 MG 14 CAPSULAS,123 - TSI 001 *,1,2.40,2.40,0.00,0.0,0.00,NaN,NaN,ADA,0,1,2,1
3,NaN,01/12/17,08:17,Contado,1,656578,MICARDIS 80 MG 28 COMP,002 - R.G. S.S.,-1,19.20,19.20,-1.92,0.0,-1.92,NaN,NaN,ADA,0,3,21,22
4,NaN,01/12/17,08:17,Contado,1,656578,MICARDIS 80 MG 28 COMP,094 - RG RE DOE,1,19.20,19.20,1.92,0.0,1.92,NaN,NaN,ADA,0,3,22,21


In [ ]:
dic_17 = dic_17.rename(columns={dic_17.columns[3]: 'operacion',
                                dic_17.columns[5]: 'Codigo',
                                dic_17.columns[6]: 'Denominacion',
                                dic_17.columns[8]: 'Cantidad',
                                dic_17.columns[10]: 'Pvp_facturado',
                                dic_17.columns[11]: 'Importe_bruto',
                                dic_17.columns[13]: 'Importe_neto',
                                dic_17.columns[15]: 'perfil',
                                dic_17.columns[18]: 'Puesto',
                                dic_17.columns[19]: 'Existencias_anteriores',
                                dic_17.columns[20]: 'Existencias_posteriores'
                                })

In [ ]:
dic_17 = dic_17.rename(columns={dic_17.columns[0]: 'columna_vacia'})
columna_vacia = 'columna_vacia'
dic_17 = dic_17.drop(columns=[columna_vacia], errors='ignore')
dic_17.head()

,Fecha,Hora,operacion,Empresa,Codigo,Denominacion,Organismo,Cantidad,Pvp,Pvp_facturado,Importe_bruto,Descuento,Importe_neto,Cliente,perfil,Vendedor,Aporta.Subv.,Puesto,Existencias_anteriores,Existencias_posteriores
0,01/12/17,08:10,Contado,1,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,091 - TSI 002,1,1.65,1.65,0.17,0.0,0.17,NaN,NaN,ADA,0,1,36,35
1,01/12/17,08:10,Contado,1,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,095 - RP RE DOE,1,1.65,1.65,0.00,0.0,0.00,NaN,NaN,ADA,0,1,37,36
2,01/12/17,08:10,Contado,1,695435,VIBRACINA 100 MG 14 CAPSULAS,123 - TSI 001 *,1,2.40,2.40,0.00,0.0,0.00,NaN,NaN,ADA,0,1,2,1
3,01/12/17,08:17,Contado,1,656578,MICARDIS 80 MG 28 COMP,002 - R.G. S.S.,-1,19.20,19.20,-1.92,0.0,-1.92,NaN,NaN,ADA,0,3,21,22
4,01/12/17,08:17,Contado,1,656578,MICARDIS 80 MG 28 COMP,094 - RG RE DOE,1,19.20,19.20,1.92,0.0,1.92,NaN,NaN,ADA,0,3,22,21


In [ ]:
dic_17 = dic_17.drop(columns=['operacion', 'Empresa', 'Cliente', 'perfil', 'Aporta.Subv.'], errors='ignore')
dic_17.head()

,Fecha,Hora,Codigo,Denominacion,Organismo,Cantidad,Pvp,Pvp_facturado,Importe_bruto,Descuento,Importe_neto,Vendedor,Puesto,Existencias_anteriores,Existencias_posteriores
0,01/12/17,08:10,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,091 - TSI 002,1,1.65,1.65,0.17,0.0,0.17,ADA,1,36,35
1,01/12/17,08:10,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,095 - RP RE DOE,1,1.65,1.65,0.00,0.0,0.00,ADA,1,37,36
2,01/12/17,08:10,695435,VIBRACINA 100 MG 14 CAPSULAS,123 - TSI 001 *,1,2.40,2.40,0.00,0.0,0.00,ADA,1,2,1
3,01/12/17,08:17,656578,MICARDIS 80 MG 28 COMP,002 - R.G. S.S.,-1,19.20,19.20,-1.92,0.0,-1.92,ADA,3,21,22
4,01/12/17,08:17,656578,MICARDIS 80 MG 28 COMP,094 - RG RE DOE,1,19.20,19.20,1.92,0.0,1.92,ADA,3,22,21


In [ ]:
dic_17['Fecha'] = pd.to_datetime(dic_17['Fecha'], format='%d/%m/%y')
dic_17['Ticket'] = dic_17.groupby(['Puesto', 'Vendedor', 'Fecha', 'Hora'])['Pvp_facturado'].transform('sum')
dic_17.head()


,Fecha,Hora,Codigo,Denominacion,Organismo,Cantidad,Pvp,Pvp_facturado,Importe_bruto,Descuento,Importe_neto,Vendedor,Puesto,Existencias_anteriores,Existencias_posteriores,Ticket
0,2017-12-01,08:10,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,091 - TSI 002,1,1.65,1.65,0.17,0.0,0.17,ADA,1,36,35,5.7
1,2017-12-01,08:10,770065,ORFIDAL 1 MG 50 COMPRIMIDOS,095 - RP RE DOE,1,1.65,1.65,0.00,0.0,0.00,ADA,1,37,36,5.7
2,2017-12-01,08:10,695435,VIBRACINA 100 MG 14 CAPSULAS,123 - TSI 001 *,1,2.40,2.40,0.00,0.0,0.00,ADA,1,2,1,5.7
3,2017-12-01,08:17,656578,MICARDIS 80 MG 28 COMP,002 - R.G. S.S.,-1,19.20,19.20,-1.92,0.0,-1.92,ADA,3,21,22,38.4
4,2017-12-01,08:17,656578,MICARDIS 80 MG 28 COMP,094 - RG RE DOE,1,19.20,19.20,1.92,0.0,1.92,ADA,3,22,21,38.4


In [ ]:
dic_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15571 entries, 0 to 15570
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Fecha                    15571 non-null  datetime64[ns]
 1   Hora                     15571 non-null  object        
 2   Codigo                   15571 non-null  int64         
 3   Denominacion             15571 non-null  object        
 4   Organismo                15571 non-null  object        
 5   Cantidad                 15571 non-null  int64         
 6   Pvp                      15571 non-null  float64       
 7   Pvp_facturado            15571 non-null  float64       
 8   Importe_bruto            15571 non-null  float64       
 9   Descuento                15571 non-null  float64       
 10  Importe_neto             15571 non-null  float64       
 11  Vendedor                 15571 non-null  object        
 12  Puesto                   15571 n

In [291]:
test = funciones.limpiar_guardar_lineas_ventas('/data/data_raw/2017/lineas_ventas_diciembre_17', '/data/data_clean/2017/lineas_ventas_diciembre_17')
test

NameError: name 'funciones' is not defined

## Resumen